<a href="https://colab.research.google.com/github/sanjay-thiyagarajan/score-predictor/blob/main/CricPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://internalapp.nptel.ac.in/contest/reference_docs/ipl_csv2.zip'
!mkdir '/content/dataset'
!unzip '/content/ipl_csv2.zip' -d '/content/dataset'

In [38]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [3]:
train_parent = []

In [4]:
DATASET_DIR = '/content/dataset/'
for file in os.listdir(DATASET_DIR):
  filename = os.path.join(DATASET_DIR, file)
  df = pd.read_csv(filename, error_bad_lines=False, header=0)
  train_parent.append(df)
train_parent = pd.concat(train_parent, axis=0, ignore_index=True)

b'Skipping line 7: expected 1 fields, saw 3\nSkipping line 10: expected 1 fields, saw 2\nSkipping line 13: expected 1 fields, saw 3\nSkipping line 26: expected 1 fields, saw 2\nSkipping line 29: expected 1 fields, saw 2\nSkipping line 30: expected 1 fields, saw 2\nSkipping line 33: expected 1 fields, saw 2\nSkipping line 37: expected 1 fields, saw 2\nSkipping line 38: expected 1 fields, saw 2\nSkipping line 39: expected 1 fields, saw 2\nSkipping line 42: expected 1 fields, saw 4\nSkipping line 47: expected 1 fields, saw 2\nSkipping line 74: expected 1 fields, saw 4\nSkipping line 78: expected 1 fields, saw 3\nSkipping line 81: expected 1 fields, saw 2\nSkipping line 84: expected 1 fields, saw 5\nSkipping line 85: expected 1 fields, saw 2\nSkipping line 88: expected 1 fields, saw 2\nSkipping line 89: expected 1 fields, saw 3\nSkipping line 92: expected 1 fields, saw 2\nSkipping line 95: expected 1 fields, saw 2\nSkipping line 924: expected 1 fields, saw 2\nSkipping line 925: expected 1 

In [6]:
train_parent[['striker', 'runs_off_bat', 'bowler']]

striker  runs_off_bat       bowler
0       Mandeep Singh           0.0   A Chandila
1       Mandeep Singh           0.0   A Chandila
2       Mandeep Singh           0.0   A Chandila
3            SE Marsh           0.0   A Chandila
4            SE Marsh           0.0   A Chandila
...               ...           ...          ...
389591        V Kohli           4.0  AB McDonald
389592        V Kohli           6.0  AB McDonald
389593        V Kohli           1.0  AB McDonald
389594       DW Steyn           0.0  AB McDonald
389595       A Kumble           1.0  AB McDonald

[389596 rows x 3 columns]

# **Checking with limited features**

In [64]:
featured_df = train_parent[['striker', 'runs_off_bat', 'bowler']]
featured_df = featured_df.dropna()
bat_encoder = LabelEncoder()
bat_encoder.fit(list(np.array(featured_df[['striker']]).flatten()))
bowl_encoder = LabelEncoder()
bowl_encoder.fit(list(np.array(featured_df[['bowler']]).flatten()))
featured_df[['striker']] = pd.DataFrame(np.array(bat_encoder.transform(featured_df[['striker']]), dtype='int32'))
featured_df[['bowler']] = pd.DataFrame(np.array(bowl_encoder.transform(featured_df[['bowler']]), dtype='int32'))
featured_df = featured_df.dropna()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [65]:
scaler = MinMaxScaler()
X_train = np.array(featured_df.drop(columns=['runs_off_bat']))
Y_train = np.array(featured_df[['runs_off_bat']])
scaler.fit_transform(X_train, Y_train)
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
train_parent = train_parent.drop(columns=['start_date', 'season', 'match_id', 'All Indian Premier League match data in CSV format'])
train_parent = pd.get_dummies(train_parent, columns=['venue', 'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler', 'wicket_type', 'player_dismissed', 'other_wicket_type', 'other_player_dismissed'])
train_parent = train_parent.fillna(0)
train_parent.head()

innings  ball  ...  player_dismissed_Yuvraj Singh  player_dismissed_Z Khan
0      1.0   0.1  ...                              0                        0
1      1.0   0.2  ...                              0                        0
2      1.0   0.3  ...                              0                        0
3      1.0   0.4  ...                              0                        0
4      1.0   0.5  ...                              0                        0

[5 rows x 2095 columns]

In [ ]:
X_train = train_parent.drop(columns=['runs_off_bat'])
Y_train = train_parent[['runs_off_bat']]
regressor = LinearRegression(fit_intercept=True)
regressor.fit(np.array(X_train), np.array(Y_train))

In [ ]:
train_parent.shape

(389596, 2095)

In [ ]:
train_parent['total_runs'] = pd.DataFrame(np.array(train_parent[['runs_off_bat']]) + np.array(train_parent[['extras']]) + np.array(train_parent[['wides']]) + np.array(train_parent[['noballs']]) + np.array(train_parent[['byes']]) + np.array(train_parent[['legbyes']]) + np.array(train_parent[['penalty']]))

In [ ]:
train_parent.head()

innings  ball  ...  player_dismissed_Z Khan  total_runs
0      1.0   0.1  ...                        0         0.0
1      1.0   0.2  ...                        0         1.0
2      1.0   0.3  ...                        0         1.0
3      1.0   0.4  ...                        0         1.0
4      1.0   0.5  ...                        0         1.0

[5 rows x 2096 columns]